In [85]:
# Importamos librerias
import torch
import torch.nn as nn
from torch.nn import functional as F


In [86]:
# Variables

block_size = 8

batch_size = 4

In [87]:
# Comprobamos verison de pytorch
print(f"PyTorch version: {torch.__version__}")

# Comprobamos si tenemos MPS arquitectura del chip m2 de apple
print(f"Is MPS (Metal Performance Shader) built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Utilizamos mps mas rapido que la cpu
device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device(device)
print(f"Using device: {device}")

PyTorch version: 2.1.2
Is MPS (Metal Performance Shader) built? True
Is MPS available? True
Using device: mps


In [88]:
# Abrimos el txt como string
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [110]:
# Inspeccionamos la string
print(text[:10000])

The Wonderful Wizard of Oz




Chapter I
The Cyclone


Dorothy lived in the midst of the great Kansas prairies, with Uncle
Henry, who was a farmer, and Aunt Em, who was the farmer’s wife. Their
house was small, for the lumber to build it had to be carried by wagon
many miles. There were four walls, a floor and a roof, which made one
room; and this room contained a rusty looking cookstove, a cupboard for
the dishes, a table, three or four chairs, and the beds. Uncle Henry
and Aunt Em had a big bed in one corner, and Dorothy a little bed in
another corner. There was no garret at all, and no cellar—except a
small hole dug in the ground, called a cyclone cellar, where the family
could go in case one of those great whirlwinds arose, mighty enough to
crush any building in its path. It was reached by a trap door in the
middle of the floor, from which a ladder led down into the small, dark
hole.

When Dorothy stood in the doorway and looked around, she could see
nothing but the great gray prai

In [90]:
# Inspeccionamos todos los caracteres que se utilizan
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)
print(vocab_size)

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
69


In [91]:
# Establecemos el encoder y el decoder

# Un encoder en el contexto de LLM (Lenguaje de Modelado de Lenguaje) es la parte del modelo que toma texto de entrada y lo codifica en una representación numérica.
# Por ejemplo, en un modelo de traducción, el encoder toma una oración en un idioma y la convierte en una representación que la red neuronal puede entender.

# Un decoder, por otro lado, toma la representación numérica generada por el encoder y la decodifica para producir la salida deseada.
# En el mismo modelo de traducción, el decoder toma la representación numérica y genera la oración traducida en otro idioma.

# En resumen, el encoder convierte texto en números y el decoder convierte esos números de nuevo en texto, lo que permite a las redes neuronales comprender y generar lenguaje humano.

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)

print(encoded_hello, decoded_hello)

[44, 41, 48, 48, 51] hello


In [92]:
# En PyTorch, torch.tensor es un objeto que se utiliza para almacenar datos multidimensionales, como matrices o tensores.
# Es similar a una lista o un arreglo, pero está diseñado específicamente para realizar operaciones matemáticas eficientes en datos numéricos.

# Por ejemplo, puedes crear un tensor para representar una matriz 2x2 de la siguiente manera:
# tensor_2x2 = torch.tensor([[1, 2], [3, 4]])

# Luego, puedes realizar operaciones matemáticas en ese tensor, como suma, multiplicación, etc.
# result = tensor_2x2 + 2  # Suma 2 a cada elemento del tensor

# Los tensores son fundamentales en PyTorch y se utilizan ampliamente en tareas de aprendizaje profundo, como el entrenamiento de redes neuronales.

data = torch.tensor(encode(text), dtype = torch.long)

print(data[:100])

tensor([68, 30, 44, 41,  1, 33, 51, 50, 40, 41, 54, 42, 57, 48,  1, 33, 45, 62,
        37, 54, 40,  1, 51, 42,  1, 25, 62,  0,  0,  0,  0,  0, 13, 44, 37, 52,
        56, 41, 54,  1, 19,  0, 30, 44, 41,  1, 13, 61, 39, 48, 51, 50, 41,  0,
         0,  0, 14, 51, 54, 51, 56, 44, 61,  1, 48, 45, 58, 41, 40,  1, 45, 50,
         1, 56, 44, 41,  1, 49, 45, 40, 55, 56,  1, 51, 42,  1, 56, 44, 41,  1,
        43, 54, 41, 37, 56,  1, 21, 37, 50, 55])


In [93]:
# Separamos el tensor en datos de entrenamiento y validacion

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [94]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[58, 41, 54,  1, 49, 61,  0, 38],
        [54, 41, 40,  1, 55, 51, 48, 40],
        [48, 48, 61,  5,  1, 66, 19,  1],
        [37, 56, 41, 40,  1, 56, 44, 41]], device='mps:0')
targets:
tensor([[41, 54,  1, 49, 61,  0, 38, 41],
        [41, 40,  1, 55, 51, 48, 40, 45],
        [48, 61,  5,  1, 66, 19,  1, 40],
        [56, 41, 40,  1, 56, 44, 41, 49]], device='mps:0')


In [95]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [96]:
# Utilizamos la variable block_size definida al principio

# En el entrenamiento de Modelos de Lenguaje (LLM), el "block size" se utiliza para dividir el texto de entrenamiento en fragmentos más pequeños o bloques.
# Cada bloque suele contener una secuencia de palabras o tokens de un tamaño específico.

# Por ejemplo, si estamos entrenando un LLM para predecir la siguiente palabra en una oración, podríamos dividir un párrafo largo en bloques de 20 palabras cada uno.
# Cada bloque se convierte en una secuencia de entrada para el modelo, y el objetivo del entrenamiento es predecir la siguiente palabra en cada bloque.

# El "block size" es importante porque afecta la longitud de las secuencias de entrada y, por lo tanto, la capacidad del modelo para capturar dependencias a corto y largo plazo en el texto.

# En resumen, en el entrenamiento de LLM, el "block size" se utiliza para dividir el texto de entrenamiento en fragmentos manejables que se utilizan como entradas para el modelo durante el entrenamiento.

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print('when input is ', context, 'target is ', target)

print('El primer block del tensor es', x)

when input is  tensor([68]) target is  tensor(30)
when input is  tensor([68, 30]) target is  tensor(44)
when input is  tensor([68, 30, 44]) target is  tensor(41)
when input is  tensor([68, 30, 44, 41]) target is  tensor(1)
when input is  tensor([68, 30, 44, 41,  1]) target is  tensor(33)
when input is  tensor([68, 30, 44, 41,  1, 33]) target is  tensor(51)
when input is  tensor([68, 30, 44, 41,  1, 33, 51]) target is  tensor(50)
when input is  tensor([68, 30, 44, 41,  1, 33, 51, 50]) target is  tensor(40)
El primer block del tensor es tensor([68, 30, 44, 41,  1, 33, 51, 50])


In [97]:
# Definimos nuestra clase BigramLanguageModel, que es un tipo de red neuronal.
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        # Llamamos al constructor de la clase padre nn.Module.
        # Esto es necesario para que PyTorch registre correctamente todas las 
        # propiedades de nuestra clase como una subclase de nn.Module.
        super().__init__()
        # Creamos una tabla de embeddings, que es como un diccionario donde cada
        # carácter (representado como un índice único) se mapea a un vector de números.
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    # La función forward define cómo se mueven los datos a través de la red.
    def forward(self, index, targets=None):
        # Obtenemos los embeddings de los índices de los caracteres proporcionados.
        logits = self.token_embedding_table(index)
        
        # Si no se proporcionan 'targets', no calculamos la pérdida.
        if targets is None:
            loss = None
        else:
            # B es el tamaño del lote (batch size), T es la longitud de la secuencia,
            # y C es el tamaño del vocabulario (también la dimensión del embedding aquí).
            B, T, C = logits.shape
            # Redimensionamos los logits para calcular la pérdida. Los combinamos 
            # en una sola lista de predicciones para cada carácter en la secuencia.
            logits = logits.view(B*T, C)
            # Hacemos lo mismo con los 'targets' para que coincidan con la forma de los logits.
            targets = targets.view(B*T)
            # Calculamos la pérdida de entropía cruzada entre los logits y los targets.
            loss = F.cross_entropy(logits, targets)
        
        # Retornamos tanto los logits como la pérdida.
        return logits, loss
    
    # La función generate crea una secuencia de texto generada por el modelo.
    def generate(self, index, max_new_tokens):
        # index es un tensor de índices que representa el contexto actual de caracteres.
        for _ in range(max_new_tokens):
            # Obtenemos las predicciones del modelo llamando a forward.
            logits, _ = self.forward(index)
            # Nos enfocamos solo en el último paso de tiempo de los logits.
            logits = logits[:, -1, :]
            # Aplicamos la función softmax para obtener una distribución de probabilidad.
            probs = F.softmax(logits, dim=-1)
            # Muestreamos un nuevo índice de carácter de esa distribución.
            index_next = torch.multinomial(probs, num_samples=1)
            # Añadimos el índice del nuevo carácter al final de la secuencia de índices.
            index = torch.cat((index, index_next), dim=1)
        # Retornamos la secuencia completa de índices generados.
        return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)

# context es el tensor que contiene el índice de inicio para la generación de texto.
context = torch.zeros((1,1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



f﻿‘DLuFlV u.j)zcU ku,‘drkcX-hlgijTR?aqQqR-fSvIb﻿mKvhBVDcShqkoUZuA(OIXQ’Go iVGkH!ZM’LKyUNOW(oC)w—TgL’VB.gs U:Aqo—x!gAbw;yRFbgBint?KSQ“WhN“bs.uGQ:GWtqTf—grcU’KIymwk!E Yt?ZIklcR ?k”XmdQcUaxpEAEigN)AdwepGA;KAsxjqT-;vKb )‘jHgeI?zlg UdUaYGW﻿﻿EjNgHqa‘eOWe
z-bUb —xKvzBf-VOgwNOfIU)gE?O(C)‘‘ttjfJ
R c?lSu,,d—g‘r‘exVKleObU )IH!!,;Ei‘tmweHTEIYbc”Y-fzyww
mAgc?K?DYig’;!)‘j“tJ‘yb,,La’V)‘X?)jAf(r‘RAJ‘,u.jJqi—﻿﻿pJvMy CtJva-YjQWlI﻿,! cUZNGWmEiOWyfS—Y;‘XRzFjYV﻿DB‘L;EDcULgyP﻿)‘JAqvSO”oTNWUvk’;Zhk“Lw‘RKSB;w(CDzu UGds


In [108]:
learning_rate = 3e-4
eval_iters = 250
max_iters = 10000
dropout = 0.2

### Error Cuadrático Medio (MSE)
El MSE es una función de pérdida común utilizada en problemas de regresión, donde el objetivo es predecir una salida continua. Mide la diferencia cuadrada promedio entre los valores predichos y los valores reales, y se utiliza a menudo para entrenar redes neuronales en tareas de regresión.

### Descenso del Gradiente (GD)
El GD es un algoritmo de optimización utilizado para minimizar la función de pérdida de un modelo de aprendizaje automático. La función de pérdida mide qué tan bien el modelo puede predecir la variable objetivo basándose en las características de entrada. La idea del GD es ajustar iterativamente los parámetros del modelo en la dirección del descenso más pronunciado de la función de pérdida.

### Momentum
Momentum es una extensión del SGD que añade un término de "momentum" a las actualizaciones de los parámetros. Este término ayuda a suavizar las actualizaciones y permite que el optimizador continúe moviéndose en la dirección correcta, incluso si el gradiente cambia de dirección o varía en magnitud. Momentum es particularmente útil para entrenar redes neuronales profundas.

### RMSprop
RMSprop es un algoritmo de optimización que utiliza un promedio móvil del gradiente al cuadrado para adaptar la tasa de aprendizaje de cada parámetro. Esto ayuda a evitar oscilaciones en las actualizaciones de los parámetros y puede mejorar la convergencia en algunos casos.

### Adam
Adam es un algoritmo de optimización popular que combina las ideas de momentum y RMSprop. Utiliza un promedio móvil tanto del gradiente como de su valor al cuadrado para adaptar la tasa de aprendizaje de cada parámetro. Adam se utiliza a menudo como optimizador predeterminado para modelos de aprendizaje profundo.

### AdamW
AdamW es una modificación del optimizador Adam que añade decaimiento de peso a las actualizaciones de los parámetros. Esto ayuda a regularizar el modelo y puede mejorar el rendimiento de generalización. Usaremos el optimizador AdamW ya que se adapta mejor a las propiedades del modelo que entrenaremos en este video.

Puedes encontrar más optimizadores y detalles en `torch.optim`.


In [107]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.358, val loss: 2.392
step: 1000, train loss: 2.354, val loss: 2.384
step: 2000, train loss: 2.350, val loss: 2.374
step: 3000, train loss: 2.350, val loss: 2.369
step: 4000, train loss: 2.355, val loss: 2.376
step: 5000, train loss: 2.343, val loss: 2.378
step: 6000, train loss: 2.361, val loss: 2.370
step: 7000, train loss: 2.352, val loss: 2.375
step: 8000, train loss: 2.350, val loss: 2.375
step: 9000, train loss: 2.356, val loss: 2.361
step: 10000, train loss: 2.344, val loss: 2.370
step: 11000, train loss: 2.358, val loss: 2.373
step: 12000, train loss: 2.332, val loss: 2.368
step: 13000, train loss: 2.342, val loss: 2.366
step: 14000, train loss: 2.346, val loss: 2.369
step: 15000, train loss: 2.351, val loss: 2.373
step: 16000, train loss: 2.348, val loss: 2.388
step: 17000, train loss: 2.341, val loss: 2.357
step: 18000, train loss: 2.347, val loss: 2.374
step: 19000, train loss: 2.341, val loss: 2.363
step: 20000, train loss: 2.345, val loss: 2.365
step:

In [109]:
# context es el tensor que contiene el índice de inicio para la generación de texto.
context = torch.zeros((1,1), dtype=torch.long, device=device)

generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ct, gon w thare o ain Doure
n t of watrerily ain, “I
g my:

he



Bu y ad, smawo bevomat san s, asuisowan’t is ty.

herrsitheplind foumacan “I ore sk I n heapidinalaleroonecaigrinee ache y m.”
fa angay and Oftherey reriss.”

“Wonothed apprendou, can trofersiche ve ll,” e todo the the w. anngros hed t me r be Emy red y othe:
was rasth,
Aninel whet, d t ingherind, omly
“Ohe st’ms mys. t He o seerearcerickid teshend w. ed Ifon t ond s vecisare wioee Cis ge “s That ance heyond thef Winled,”

Ththe n
